In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

In [2]:
all_data = pd.read_csv("data/processed/merged_data.csv")


C:\Users\ParkJaehyung\AppData\Local\Temp\ipykernel_13916\109851504.py:1: DtypeWarning: Columns (5,8,15,38) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv("Full-Dataset/processed/merged_data.csv")


In [3]:
def getRNN_dataset(df):
    columns = ["http.data.key", "http.data.value", "Attack_type"]
    columns = [_ for _ in columns if _ in df.columns]
    return df[list(columns)]
def get_logistic_dataset(df):
    columns = ["http.data.key", "http.data.value"]
    columns = [_ for _ in columns if _ in df.columns]
    return df.drop(columns, axis=1)

In [4]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# 데이터를 로드합니다.
df = all_data

# LabelEncoder 클래스를 생성합니다.
encoder = LabelEncoder()
df["Attack_type"] = encoder.fit_transform(df["Attack_type"])

# 데이터를 훈련 세트와 테스트 세트로 나눕니다.
splitted_data = train_test_split(df, df["Attack_type"], test_size=0.2)
X_train, X_test, y_train, y_test = splitted_data
RNN_X_train, RNN_X_test = [getRNN_dataset(_) for _ in [X_train, X_test]]
logistic_X_train, logistic_X_test = [get_logistic_dataset(_) for _ in [X_train, X_test]]

In [5]:
encoder.classes_


array(['Password', 'SQL_injection', 'Uploading', 'XSS'], dtype=object)

In [6]:
# RNN을 사용하여 요청의 URI 키와 값을 분류하는 모델을 학습합니다.
rnn_model = tf.keras.Sequential([
  tf.keras.layers.Embedding(5000, 128),
  tf.keras.layers.Conv1D(64, 3, padding='same'),
  tf.keras.layers.LSTM(128),
  tf.keras.layers.Dense(4, activation="softmax")
])

rnn_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
rnn_model.fit(RNN_X_train, y_train, epochs=10)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [ ]:
# 로지스틱 회귀를 사용하여 나머지 데이터를 분류하는 모델을 학습합니다.
logistic_regression_model = tf.keras.Sequential([
  tf.keras.layers.Dense(128, activation="relu"),
  tf.keras.layers.Dense(4, activation="softmax")
])

logistic_regression_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
logistic_regression_model.fit(X_train, y_train, epochs=10)

In [ ]:
# 두 모델의 예측을 결합하여 최종 예측을 만듭니다.
predictions = rnn_model.predict(RNN_X_test) * 0.5 + logistic_regression_model.predict(logistic_X_test) * 0.5

# 모델의 성능을 평가합니다.
accuracy = tf.metrics.accuracy(labels=y_test, predictions=predictions)
print("Accuracy:", accuracy)